In [84]:
import pandas as pd
import numpy as np
from datetime import datetime, time

# 기존 데이터를 딕셔너리 형태로 생성
data = {
    'Name': ['John', 'John', 'John', 'John', 'John', 'John', 'John', 'John', 'John', 'John', 
             'John', 'John', 'John', 'John', 'John', 'John', 'John', 'John', 'John', 'John', 
             'John', 'John', 'John', 'John', 'John', 'John', 'John', 'John', 'Kane', 'Kane', 
             'Kane', 'Kane', 'Kane', 'Kane', 'Kane', 'Kane', 'Kane', 'Kane', 'Kane', 'Kane', 
             'Kane', 'Kane', 'Kane', 'Kane', 'Kane', 'Kane', 'Kane', 'Kane', 'Kane', 'Kane', 
             'Kane', 'Kane', 'Kane', 'Kane', 'Kane', 'Kane'],
    'Date': ['1/12/25', '1/12/25', '1/13/25', '1/13/25', '1/14/25', '1/14/25', '1/15/25', 
             '1/15/25', '1/16/25', '1/16/25', '1/17/25', '1/17/25', '1/18/25', '1/18/25', 
             '1/19/25', '1/19/25', '1/20/25', '1/20/25', '1/21/25', '1/21/25', '1/22/25', 
             '1/22/25', '1/23/25', '1/23/25', '1/24/25', '1/24/25', '1/25/25', '1/25/25', 
             '1/12/25', '1/12/25', '1/13/25', '1/13/25', '1/14/25', '1/14/25', '1/15/25', 
             '1/15/25', '1/16/25', '1/16/25', '1/17/25', '1/17/25', '1/17/25', '1/17/25', 
             '1/18/25', '1/18/25', '1/19/25', '1/19/25', '1/20/25', '1/20/25', '1/21/25', 
             '1/21/25', '1/22/25', '1/22/25', '1/23/25', '1/23/25', '1/24/25', '1/24/25'],
    'Card Reader': ['Entry', 'Exit', 'Entry', 'Exit', 'Entry', 'Exit', 'Entry', 'Exit', 'Entry', 
                    'Exit', 'Entry', 'Exit', 'Entry', 'Exit', 'Entry', 'Exit', 'Entry', 'Exit', 
                    'Entry', 'Exit', 'Entry', 'Exit', 'Entry', 'Exit', 'Entry', 'Exit', 'Entry', 
                    'Exit', 'Entry', 'Exit', 'Entry', 'Exit', 'Entry', 'Exit', 'Entry', 'Exit', 
                    'Entry', 'Exit', 'Entry', 'Exit', 'Entry', 'Exit', 'Entry', 'Exit', 'Entry', 
                    'Exit', 'Entry', 'Exit', 'Entry', 'Exit', 'Entry', 'Exit', 'Entry', 'Exit', 
                    'Entry'],
    'Time': ['0:00', '0:00', '0:00', '0:00', '0:00', '0:00', '0:00', '0:00', '7:00', '16:00', 
             '7:00', '16:00', '0:00', '0:00', '0:00', '0:00', '0:00', '0:00', '0:00', '0:00', 
             '0:00', '0:00', '0:00', '0:00', '0:00', '0:00', '0:00', '0:00', '0:00', '0:00', 
             '0:00', '0:00', '0:00', '0:00', '0:00', '0:00', '7:00', '16:00', '7:00', '13:00', 
             '14:00', '16:00', '0:00', '0:00', '0:00', '0:00', '0:00', '0:00', '0:00', '0:00', 
             '0:00', '0:00', '0:00', '0:00', '0:00']
}

# 데이터프레임 생성
df = pd.DataFrame(data)

def optimized_calculate_hours_bigdata(df):
    # Convert date and time into a single datetime column
    df['Datetime'] = pd.to_datetime(df['Date'] + ' ' + df['Time'], format='%m/%d/%y %H:%M')

    # Sort the data (if not already sorted)
    df = df.sort_values(['Name', 'Date', 'Datetime'])

    # Categorize columns to save memory
    df['Card Reader'] = df['Card Reader'].astype('category')
    df['Name'] = df['Name'].astype('category')

    # Create pairing for Entry and Exit using shift
    df['Next_Datetime'] = df['Datetime'].shift(-1)
    df['Next_Name'] = df['Name'].shift(-1)
    df['Next_Date'] = df['Date'].shift(-1)

    # Filter only valid Entry-Exit pairs (same Name and Date)
    df['Valid_Pair'] = (df['Card Reader'] == 'Entry') & (df['Next_Name'] == df['Name']) & (df['Next_Date'] == df['Date'])

    # Calculate duration for valid pairs
    df['Duration'] = np.where(df['Valid_Pair'], (df['Next_Datetime'] - df['Datetime']).dt.total_seconds() / 3600, 0)

    # Core time boundaries
    core_start = time(10, 0)
    core_end = time(15, 0)

    def calculate_core_hours(start, end):
        """Calculate overlap with core hours."""
        core_start_dt = datetime.combine(start.date(), core_start)
        core_end_dt = datetime.combine(start.date(), core_end)
        overlap_start = max(start, core_start_dt)
        overlap_end = min(end, core_end_dt)
        return max(0, (overlap_end - overlap_start).total_seconds() / 3600)

    # Vectorized calculation of core hours
    df['Core_Hours'] = np.where(
        df['Valid_Pair'],
        df.apply(lambda row: calculate_core_hours(row['Datetime'], row['Next_Datetime']), axis=1),
        0
    )

    # Aggregate results by Name and Date
    summary = df.groupby(['Name', 'Date']).agg(
        Total_Hours=('Duration', 'sum'),
        Min_Entry=('Datetime', 'min'),
        Max_Exit=('Datetime', lambda x: x.max() if len(x) > 1 else x.iloc[-1]),
        Core_Time_Hours=('Core_Hours', 'sum')
    ).reset_index()

    # Add Five Hours or More flag
    summary['Five_Hours_or_More'] = summary['Core_Time_Hours'] >= 5

    # Format Min_Entry and Max_Exit as time only
    summary['Min_Entry'] = summary['Min_Entry'].dt.time
    summary['Max_Exit'] = summary['Max_Exit'].dt.time

    return summary

# Example usage
# df = pd.read_csv('your_large_dataset.csv')  # Load your big dataset




ValueError: arrays must all be same length

In [82]:
data = {
    'Name': ['John', 'John', 'John', 'John', 'John', 'John', 'John', 'John', 'John', 'John', 'John', 'John', 'John', 'John', 'John', 'John', 'John', 'John', 'John', 'John', 'John', 'John', 'John', 'John', 'John', 'John', 'John', 'John', 'Kane', 'Kane', 'Kane', 'Kane', 'Kane', 'Kane', 'Kane', 'Kane', 'Kane', 'Kane', 'Kane', 'Kane', 'Kane', 'Kane', 'Kane', 'Kane', 'Kane', 'Kane', 'Kane', 'Kane', 'Kane', 'Kane', 'Kane', 'Kane', 'Kane', 'Kane'],
    'Date': ['1/12/25', '1/12/25', '1/13/25', '1/13/25', '1/14/25', '1/14/25', '1/15/25', '1/15/25', '1/16/25', '1/16/25', '1/17/25', '1/17/25', '1/18/25', '1/18/25', '1/19/25', '1/19/25', '1/20/25', '1/20/25', '1/21/25', '1/21/25', '1/22/25', '1/22/25', '1/23/25', '1/23/25', '1/24/25', '1/24/25', '1/25/25', '1/25/25', '1/12/25', '1/12/25', '1/13/25', '1/13/25', '1/14/25', '1/14/25', '1/15/25', '1/15/25', '1/16/25', '1/16/25', '1/17/25', '1/17/25', '1/18/25', '1/18/25', '1/19/25', '1/19/25', '1/20/25', '1/20/25', '1/21/25', '1/21/25', '1/22/25', '1/22/25', '1/23/25', '1/23/25', '1/24/25', '1/24/25', '1/25/25', '1/25/25'],
    'Card Reader': ['Entry', 'Exit', 'Entry', 'Exit', 'Entry', 'Exit', 'Entry', 'Exit', 'Entry', 'Exit', 'Entry', 'Exit', 'Entry', 'Exit', 'Entry', 'Exit', 'Entry', 'Exit', 'Entry', 'Exit', 'Entry', 'Exit', 'Entry', 'Exit', 'Entry', 'Exit', 'Entry', 'Exit', 'Entry', 'Exit', 'Entry', 'Exit', 'Entry', 'Exit', 'Entry', 'Exit', 'Entry', 'Exit', 'Entry', 'Exit', 'Entry', 'Exit', 'Entry', 'Exit', 'Entry', 'Exit', 'Entry', 'Exit', 'Entry', 'Exit', 'Entry', 'Exit', 'Entry', 'Exit', 'Entry', 'Exit'],
    'Time': ['0:00', '0:00', '0:00', '0:00', '0:00', '0:00', '0:00', '0:00', '0:00', '0:00', '0:00', '0:00', '0:00', '0:00', '0:00', '0:00', '0:00', '0:00', '0:00', '0:00', '0:00', '0:00', '0:00', '0:00', '0:00', '0:00', '0:00', '0:00', '7:00', '16:00', '7:00', '16:00', '0:00', '0:00', '0:00', '0:00', '0:00', '0:00', '0:00', '0:00', '0:00', '0:00', '0:00', '0:00', '0:00', '0:00', '0:00', '0:00', '0:00', '0:00', '7:00', '16:00', '7:00', '13:00', '0:00', '0:00']
}


# Create the DataFrame
df = pd.DataFrame(data)

ValueError: arrays must all be same length

In [79]:
data

{'Name': ['John',
  'John',
  'John',
  'John',
  'John',
  'John',
  'John',
  'John',
  'John',
  'John',
  'John',
  'John',
  'John',
  'John',
  'John',
  'John',
  'John',
  'John',
  'John',
  'John',
  'John',
  'John',
  'John',
  'John',
  'John',
  'John',
  'John',
  'John',
  'Kane',
  'Kane',
  'Kane',
  'Kane',
  'Kane',
  'Kane',
  'Kane',
  'Kane',
  'Kane',
  'Kane',
  'Kane',
  'Kane',
  'Kane',
  'Kane',
  'Kane',
  'Kane',
  'Kane',
  'Kane',
  'Kane',
  'Kane',
  'Kane',
  'Kane',
  'Kane',
  'Kane',
  'Kane',
  'Kane'],
 'Date': ['1/12/25',
  '1/12/25',
  '1/13/25',
  '1/13/25',
  '1/14/25',
  '1/14/25',
  '1/15/25',
  '1/15/25',
  '1/16/25',
  '1/16/25',
  '1/17/25',
  '1/17/25',
  '1/18/25',
  '1/18/25',
  '1/19/25',
  '1/19/25',
  '1/20/25',
  '1/20/25',
  '1/21/25',
  '1/21/25',
  '1/22/25',
  '1/22/25',
  '1/23/25',
  '1/23/25',
  '1/24/25',
  '1/24/25',
  '1/25/25',
  '1/25/25',
  '1/12/25',
  '1/12/25',
  '1/13/25',
  '1/13/25',
  '1/14/25',
  '1/14/25',
 

In [73]:
result = optimized_calculate_hours_bigdata(df)
result

,Name,Date,Total_Hours,Min_Entry,Max_Exit,Core_Time_Hours,Five_Hours_or_More
0,John,1/12/25,0.0,00:00:00,00:00:00,0.0,False
1,John,1/13/25,0.0,00:00:00,00:00:00,0.0,False
2,John,1/14/25,0.0,00:00:00,00:00:00,0.0,False
3,John,1/15/25,0.0,00:00:00,00:00:00,0.0,False
4,John,1/16/25,9.0,07:00:00,16:00:00,5.0,True
5,John,1/17/25,9.0,07:00:00,16:00:00,5.0,True
6,John,1/18/25,0.0,00:00:00,00:00:00,0.0,False
7,John,1/19/25,0.0,00:00:00,00:00:00,0.0,False
8,John,1/20/25,0.0,00:00:00,00:00:00,0.0,False
9,John,1/21/25,0.0,00:00:00,00:00:00,0.0,False


In [65]:
import pandas as pd

# Dataset 1
data1 = {
    "NAME": ["John", "Kane"],
    "PayPeriod_NO": ["2024-23", "2024-23"],
    "Remote Status": ["Approved", ""],
    "Telework Status": ["", "Approved"],
    "Type of Telework Plan": ["", "Routine"],
    "Sunday1": ["", ""],
    "Monday1": ["", "YES"],
    "Tuesday1": ["", "YES"],
    "Wednesday1": ["", "YES"],
    "Thursday1": ["", "NO"],
    "Friday1": ["", "NO"],
    "Saturday1": ["", ""],
    "Sunday2": ["", ""],
    "Monday2": ["", "YES"],
    "Tuesday2": ["", "YES"],
    "Wednesday2": ["", "YES"],
    "Thursday2": ["", "NO"],
    "Friday2": ["", "NO"],
    "Saturday2": ["", ""]
}

# Dataset 2
data2 = {
    "PayPeriod_NO": ["2024-23"] * 14,
    "Date": [
        "1/12/25", "1/13/25", "1/14/25", "1/15/25", "1/16/25", "1/17/25", "1/18/25",
        "1/19/25", "1/20/25", "1/21/25", "1/22/25", "1/23/25", "1/24/25", "1/25/25"
    ],
    "Day": [
        "Sunday1", "Monday1", "Tuesday1", "Wednesday1", "Thursday1", "Friday1", "Saturday1",
        "Sunday2", "Monday2", "Tuesday2", "Wednesday2", "Thursday2", "Friday2", "Saturday2"
    ]
}

# Convert to pandas DataFrames
df1 = pd.DataFrame(data1)
df2 = pd.DataFrame(data2)

def optimized_join_large_datasets(df1, df2):
    # Convert columns to categorical for memory efficiency
    df1["NAME"] = df1["NAME"].astype("category")
    df1["PayPeriod_NO"] = df1["PayPeriod_NO"].astype("category")

    df2["PayPeriod_NO"] = df2["PayPeriod_NO"].astype("category")
    df2["Day"] = df2["Day"].astype("category")

    # Ensure the "Date" column in df2 is formatted as datetime
    df2["Date"] = pd.to_datetime(df2["Date"], format="%m/%d/%y")

    # Melt df1 to align days with their telework values
    df1_melted = df1.melt(
        id_vars=["NAME", "PayPeriod_NO", "Remote Status", "Telework Status", "Type of Telework Plan"],
        var_name="Day",
        value_name="InTelework"
    )

    # Filter out rows where InTelework is empty to reduce unnecessary joins
    #df1_melted = df1_melted[df1_melted["InTelework"].notna() & (df1_melted["InTelework"] != "")]

    # Perform the join on PayPeriod_NO and Day
    result = pd.merge(
        df1_melted, df2, how="left", left_on=["PayPeriod_NO", "Day"], right_on=["PayPeriod_NO", "Day"]
    )

    # Sort the joined dataset by NAME and Date
    result_sorted = result.sort_values(by=["NAME", "Date"]).reset_index(drop=True)

    # Reorder columns
    result_sorted = result_sorted[
        ["NAME", "PayPeriod_NO", "Remote Status", "Telework Status", "Type of Telework Plan", "Date", "Day", "InTelework"]
    ]

    return result_sorted




In [66]:
df1

,NAME,PayPeriod_NO,Remote Status,Telework Status,Type of Telework Plan,Sunday1,Monday1,Tuesday1,Wednesday1,Thursday1,Friday1,Saturday1,Sunday2,Monday2,Tuesday2,Wednesday2,Thursday2,Friday2,Saturday2
0,John,2024-23,Approved,,,,,,,,,,,,,,,,
1,Kane,2024-23,,Approved,Routine,,YES,YES,YES,NO,NO,,,YES,YES,YES,NO,NO,


In [67]:
df2

,PayPeriod_NO,Date,Day
0,2024-23,1/12/25,Sunday1
1,2024-23,1/13/25,Monday1
2,2024-23,1/14/25,Tuesday1
3,2024-23,1/15/25,Wednesday1
4,2024-23,1/16/25,Thursday1
5,2024-23,1/17/25,Friday1
6,2024-23,1/18/25,Saturday1
7,2024-23,1/19/25,Sunday2
8,2024-23,1/20/25,Monday2
9,2024-23,1/21/25,Tuesday2


In [68]:
# Example usage
result_optimized = optimized_join_large_datasets(df1, df2)
result_optimized

,NAME,PayPeriod_NO,Remote Status,Telework Status,Type of Telework Plan,Date,Day,InTelework
0,John,2024-23,Approved,,,2025-01-12,Sunday1,
1,John,2024-23,Approved,,,2025-01-13,Monday1,
2,John,2024-23,Approved,,,2025-01-14,Tuesday1,
3,John,2024-23,Approved,,,2025-01-15,Wednesday1,
4,John,2024-23,Approved,,,2025-01-16,Thursday1,
5,John,2024-23,Approved,,,2025-01-17,Friday1,
6,John,2024-23,Approved,,,2025-01-18,Saturday1,
7,John,2024-23,Approved,,,2025-01-19,Sunday2,
8,John,2024-23,Approved,,,2025-01-20,Monday2,
9,John,2024-23,Approved,,,2025-01-21,Tuesday2,
